In [17]:
import pandas as pd
import os
from datetime import datetime as dt

import itertools
import networkx as nx
import matplotlib.pyplot as plt

import plotly.graph_objects as go
from collections import Counter

import xlwings as xw
import copy

base = os.getcwd() + "\\"

In [18]:
app = xw.App()
wb = app.books.open(base + "IPL_2024.xlsx")

wb.api.RefreshAll()
wb.close()
app.kill()

In [19]:
matches_raw =      pd.read_excel(base + "IPL_2024.xlsx", sheet_name = 'League Matches')
points_table_raw = pd.read_excel(base + "IPL_2024.xlsx", sheet_name = 'Points Table')
mapping      = pd.read_excel(base + "IPL_2024.xlsx", sheet_name = 'Mapping')

display(mapping)

,Team Full Name,Team Half Name,Team Abbr
0,Chennai Super Kings,Super Kings,CSK
1,Punjab Kings,Kings,PBKS
2,Kolkata Knight Riders,Knight Riders,KKR
3,Rajasthan Royals,Royals,RR
4,Gujarat Titans,Titans,GT
5,Royal Challengers Bengaluru,Royal Challengers,RCB
6,Sunrisers Hyderabad,Sunrisers,SRH
7,Lucknow Super Giants,Super Giants,LSG
8,Delhi Capitals,Capitals,DC
9,Mumbai Indians,Indians,MI


In [20]:
matches = matches_raw.copy()
matches.columns = [i.replace("\n"," ") for i in matches.columns]
matches['DATETIME'] = pd.to_datetime(matches['DATE'] + " " + matches['TIME'])

matches = matches.merge(mapping[['Team Full Name','Team Abbr']].rename(columns = {'Team Full Name':'HOME', 'Team Abbr':'HOME_TEAM'}),on = 'HOME', how = 'left')
matches = matches.merge(mapping[['Team Full Name','Team Abbr']].rename(columns = {'Team Full Name':'AWAY', 'Team Abbr':'AWAY_TEAM'}),on = 'AWAY', how = 'left')

curr_date = dt(2024,5,5,19)
curr_date = dt.today()
# .replace(hour = 0)

print(curr_date)

future_matches = matches[matches['DATETIME']>=curr_date]
display(future_matches)

2024-05-11 14:15:28.481662


,MATCH NO,MATCH DAY,DATE,DAY,TIME,HOME,AWAY,STADIUM,DATETIME,HOME_TEAM,AWAY_TEAM
59,60,51,11-May-24,Sat,7:30PM,Kolkata Knight Riders,Mumbai Indians,Kolkata,2024-05-11 19:30:00,KKR,MI
60,61,52,12-May-24,Sun,3:30PM,Chennai Super Kings,Rajasthan Royals,Chennai,2024-05-12 15:30:00,CSK,RR
61,62,52,12-May-24,Sun,7:30PM,Royal Challengers Bengaluru,Delhi Capitals,Bengaluru,2024-05-12 19:30:00,RCB,DC
62,63,53,13-May-24,Mon,7:30PM,Gujarat Titans,Kolkata Knight Riders,Ahmedabad,2024-05-13 19:30:00,GT,KKR
63,64,54,14-May-24,Tue,7:30PM,Delhi Capitals,Lucknow Super Giants,Delhi,2024-05-14 19:30:00,DC,LSG
64,65,55,15-May-24,Wed,7:30PM,Rajasthan Royals,Punjab Kings,Guwahati,2024-05-15 19:30:00,RR,PBKS
65,66,56,16-May-24,Thu,7:30PM,Sunrisers Hyderabad,Gujarat Titans,Hyderabad,2024-05-16 19:30:00,SRH,GT
66,67,57,17-May-24,Fri,7:30PM,Mumbai Indians,Lucknow Super Giants,Mumbai,2024-05-17 19:30:00,MI,LSG
67,68,58,18-May-24,Sat,7:30PM,Royal Challengers Bengaluru,Chennai Super Kings,Bengaluru,2024-05-18 19:30:00,RCB,CSK
68,69,59,19-May-24,Sun,3:30PM,Sunrisers Hyderabad,Punjab Kings,Hyderabad,2024-05-19 15:30:00,SRH,PBKS


In [21]:
points_table = points_table_raw.copy()
points_table = points_table.merge(mapping[['Team Half Name','Team Abbr']].rename(columns = {'Team Half Name':'TEAM', 'Team Abbr':'TEAM_NAME'}),on = 'TEAM', how = 'left')
display(points_table)

# points_table.loc[points_table['TEAM_NAME']=='SRH', 'PTS']+=2
# display(points_table)


,TEAM,MATCHES,WON,LOST,PTS,NRR,TEAM_NAME
0,Knight Riders,11,8,3,16,1.453,KKR
1,Royals,11,8,3,16,0.476,RR
2,Sunrisers,12,7,5,14,0.406,SRH
3,Super Kings,12,6,6,12,0.491,CSK
4,Capitals,12,6,6,12,-0.316,DC
5,Super Giants,12,6,6,12,-0.769,LSG
6,Royal Challengers,12,5,7,10,0.217,RCB
7,Titans,12,5,7,10,-1.063,GT
8,Indians,12,4,8,8,-0.212,MI
9,Kings,12,4,8,8,-0.423,PBKS


In [22]:
class Team:
    def __init__(self, name, points=0):
        self.name = name
        self.points = points

class Match:
    def __init__(self, team1, team2):
        self.team1 = team1
        self.team2 = team2

team_pts = points_table[['TEAM_NAME','PTS']].set_index('TEAM_NAME').to_dict()['PTS']
teams = {t:Team(t,p) for t,p in team_pts.items()}

In [23]:
future_match_dict = future_matches.set_index('MATCH NO').T.to_dict()
match_dict = {k:Match(teams[v['HOME_TEAM']], teams[v['AWAY_TEAM']]) for k,v in future_match_dict.items()}

In [24]:
print(list([i.name for i in teams.values()]), list(match_dict.values()))


['KKR', 'RR', 'SRH', 'CSK', 'DC', 'LSG', 'RCB', 'GT', 'MI', 'PBKS'] [<__main__.Match object at 0x00000285378812B0>, <__main__.Match object at 0x0000028537881430>, <__main__.Match object at 0x00000285378812E0>, <__main__.Match object at 0x000002853787DAF0>, <__main__.Match object at 0x000002853787DA60>, <__main__.Match object at 0x000002853787DBE0>, <__main__.Match object at 0x000002853787DB20>, <__main__.Match object at 0x000002853787DF10>, <__main__.Match object at 0x000002853787DC40>, <__main__.Match object at 0x000002853787D2B0>, <__main__.Match object at 0x000002853787D8B0>]


In [25]:
class MatchOutcome:
    def __init__(self, match, result, winners = [], parent=None):
        self.match = match
        self.result = result
        self.parent = parent
        self.children = []
        self.winners = winners
        # self.width = len(self.children)

        # if parent:
        #     self.width = result + 1
            
        #     self.level = self.parent.level + 1
        # else:
        #     self.level = 0
        #     self.width = result

        # print(self.level, self.width)

        # if match:
        #     print(f"{match.team1.name} v {match.team2.name} \t W - ",[match.team1.name,match.team2.name][result])
        #     if result==0: pt_tbl[match.team1.name]+=2
        #     else: pt_tbl[match.team2.name]+=2



        # print(sorted(tuple(pt_tbl.items()), key = lambda x: -x[1]))
        # self.pt_tbl = pt_tbl

def generate_match_tree(matches):
    root = MatchOutcome(None, None)
    generate_children(root, matches)
    return root

def generate_children(parent_node, remaining_matches):
    if not remaining_matches:
        return
    
    match = remaining_matches[0]
    for result in [0, 1]: 
        print(parent_node.winners)
        child_node = MatchOutcome(match, result,  parent_node.winners + [[match.team1.name,match.team2.name][result]], parent_node)
        parent_node.children.append(child_node)
        generate_children(child_node, remaining_matches[1:])



# print(points_dict)
root = generate_match_tree(list(match_dict.values()))
        

[]
['KKR']
['KKR', 'CSK']
['KKR', 'CSK', 'RCB']
['KKR', 'CSK', 'RCB', 'GT']
['KKR', 'CSK', 'RCB', 'GT', 'DC']
['KKR', 'CSK', 'RCB', 'GT', 'DC', 'RR']
['KKR', 'CSK', 'RCB', 'GT', 'DC', 'RR', 'SRH']
['KKR', 'CSK', 'RCB', 'GT', 'DC', 'RR', 'SRH', 'MI']
['KKR', 'CSK', 'RCB', 'GT', 'DC', 'RR', 'SRH', 'MI', 'RCB']
['KKR', 'CSK', 'RCB', 'GT', 'DC', 'RR', 'SRH', 'MI', 'RCB', 'SRH']
['KKR', 'CSK', 'RCB', 'GT', 'DC', 'RR', 'SRH', 'MI', 'RCB', 'SRH']
['KKR', 'CSK', 'RCB', 'GT', 'DC', 'RR', 'SRH', 'MI', 'RCB']
['KKR', 'CSK', 'RCB', 'GT', 'DC', 'RR', 'SRH', 'MI', 'RCB', 'PBKS']
['KKR', 'CSK', 'RCB', 'GT', 'DC', 'RR', 'SRH', 'MI', 'RCB', 'PBKS']
['KKR', 'CSK', 'RCB', 'GT', 'DC', 'RR', 'SRH', 'MI']
['KKR', 'CSK', 'RCB', 'GT', 'DC', 'RR', 'SRH', 'MI', 'CSK']
['KKR', 'CSK', 'RCB', 'GT', 'DC', 'RR', 'SRH', 'MI', 'CSK', 'SRH']
['KKR', 'CSK', 'RCB', 'GT', 'DC', 'RR', 'SRH', 'MI', 'CSK', 'SRH']
['KKR', 'CSK', 'RCB', 'GT', 'DC', 'RR', 'SRH', 'MI', 'CSK']
['KKR', 'CSK', 'RCB', 'GT', 'DC', 'RR', 'SRH', 'MI', 

In [26]:
def evaluate_condition(node, points_dict, check_team):
    # Example condition: Check if the leaf node's match result is a tie
    winning_pts = dict(Counter(node.winners))
    for t in points_dict.keys():
        if t not in winning_pts.keys():
            winning_pts[t] = 0
        else: winning_pts[t] = winning_pts[t]*2

            
    all_points = list({t: points_dict[t] + winning_pts[t] for t in points_dict.keys()}.items())
    
    all_points = sorted(list(all_points), key = lambda x: -x[-1])
    top4 = all_points[:4]

    if all([t in [i[0] for i in top4] for t in check_team]): 
        print(all_points)
        return True
    return False
    # return node.match is not None and node.match.team1.name == "Team A" and node.result == 1  # Replace with your condition

def prune_tree(root, points_dict, check_team):
    win_list = []
    stack = [root]
    leaf_node = []
    while stack:
        node = stack.pop()
        if not node.children:  # Leaf node
            leaf_node.append(node)
            if not evaluate_condition(node, dict(points_dict), check_team):
                # Prune subtree by removing all children
                node.parent.children.remove(node)
            else:
                win_list.append(node)
        else:
            stack.extend(node.children)
    return root, win_list, leaf_node

points_table['PTxNRR'] = points_table['PTS']*points_table['NRR'].apply(abs)
points_dict = points_table.set_index('TEAM_NAME').to_dict()['PTS']
pruned_tree, win_leaf, leaf_nodes = prune_tree(copy.deepcopy(root), points_dict, ["RCB"])

In [27]:
print(len(win_leaf)/len(leaf_nodes)*100)

0.0


In [28]:
def evaluate_condition(node, points_dict, check_team):
    # Example condition: Check if the leaf node's match result is a tie
    winning_pts = dict(Counter(node.winners))
    for t in points_dict.keys():
        if t not in winning_pts.keys():
            winning_pts[t] = 0
        else: winning_pts[t] = winning_pts[t]*2

            
    all_points = list({t: points_dict[t] + winning_pts[t] for t in points_dict.keys()}.items())
    
    all_points = [i[0] for i in sorted(list(all_points), key = lambda x: -x[-1])]

    # pt_rank = {pt:k+1 for k,pt in enumerate(sorted(set([i[-1] for i in all_points]))[::-1])}
    # ranks = {t: pt_rank[pt] for t,pt in all_points}
    # return ranks[check_team]
    return all_points.index(check_team)+1
    # top4 = all_points[:3]

    # if all([t in [i[0] for i in top4] for t in check_team]): 
    #     print(all_points)
    #     return True
    # return False

def prune_tree(root, points_dict, check_team):
    # print(check_team)
    point_list = []
    stack = [root]
    leaf_node = []
    while stack:
        node = stack.pop()
        if not node.children:  # Leaf node
            leaf_node.append(node)
            point_list.append(evaluate_condition(node, dict(points_dict), check_team))
            # if not evaluate_condition(node, dict(points_dict), check_team):
            #     # Prune subtree by removing all children
            #     node.parent.children.remove(node)
            # else:
            #     win_list.append(node)
        else:
            stack.extend(node.children)

    point_dict = dict(Counter(point_list))
    point_dict = {k:v/len(point_list) for k,v in point_dict.items()}
    return point_dict

points_dict = points_table.set_index('TEAM_NAME').to_dict()['PTS']

team_points = {}
for t in mapping['Team Abbr'].unique():
    print(t)
    team_points[t] =  prune_tree(copy.deepcopy(root), points_dict, t)

CSK
PBKS
KKR
RR
GT
RCB
SRH
LSG
DC
MI


In [29]:
points_dict

{'KKR': 16,
 'RR': 16,
 'SRH': 14,
 'CSK': 12,
 'DC': 12,
 'LSG': 12,
 'RCB': 10,
 'GT': 10,
 'MI': 8,
 'PBKS': 8}

In [30]:
team_points2 = {k:pd.DataFrame(v, index = [k]) for k,v in team_points.items()}
perc = pd.concat(team_points2.values()).T.sort_index().T.rename_axis('RANK')
perc.loc[:,'Top 4'] = perc.iloc[:,:4].sum(axis=1)
# perc.loc['Min Rank',:] = perc.iloc[:,:].max()
# perc.loc['Total',:] = perc.sum(axis=0)
# print(perc.columns)
perc = perc.T[list(perc[['Top 4']].sort_values(by = 'Top 4', ascending = False).index.tolist())]
perc = perc.fillna(0).applymap(lambda x: f'{x:.2%}')



# Define a function to apply conditional formatting
def highlight_cells(val):
    # print(val)
    return f'background-color: rgba(0, 100, 255, {val})'  # Green color with alpha based on value



# Apply styling to the DataFrame
styled_df = perc.style.applymap(highlight_cells)



# Render styled DataFrame
styled_df

RANK,KKR,RR,SRH,CSK,DC,LSG,PBKS,GT,RCB,MI
1,62.50%,37.50%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%
2,34.38%,59.38%,6.25%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%
3,3.12%,3.12%,78.12%,6.25%,4.69%,4.69%,0.00%,0.00%,0.00%,0.00%
4,0.00%,0.00%,12.50%,43.75%,25.00%,18.75%,0.00%,0.00%,0.00%,0.00%
5,0.00%,0.00%,3.12%,27.34%,35.94%,23.44%,0.00%,0.78%,9.38%,0.00%
6,0.00%,0.00%,0.00%,14.84%,18.75%,42.19%,0.00%,11.72%,12.50%,0.00%
7,0.00%,0.00%,0.00%,7.03%,12.50%,9.38%,1.17%,24.22%,44.14%,1.56%
8,0.00%,0.00%,0.00%,0.78%,3.12%,1.56%,7.42%,52.34%,25.39%,9.38%
9,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,25.39%,9.38%,7.42%,57.81%
10,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,66.02%,1.56%,1.17%,31.25%


In [ ]:
team_points2 = {k:pd.DataFrame(v, index = [k]) for k,v in team_points.items()}
perc = pd.concat(team_points2.values()).T.sort_index().T.rename_axis('RANK')
perc.loc[:,'Top 4'] = perc.iloc[:,:4].sum(axis=1)
perc

In [ ]:
perc.T[['Top 4']].sort_values(by = 'Top 4', ascending = False).index



In [ ]:
def evaluate_condition(node, points_dict, check_team):
    # Example condition: Check if the leaf node's match result is a tie
    winning_pts = dict(Counter(node.winners))
    for t in points_dict.keys():
        if t not in winning_pts.keys():
            winning_pts[t] = 0
        else: winning_pts[t] = winning_pts[t]*2

            
    all_points = list({t: points_dict[t] + winning_pts[t] for t in points_dict.keys()}.items())
    
    all_points = [i[0] for i in sorted(list(all_points), key = lambda x: -x[-1])]

    return all_points.index(check_team)+1
    # top4 = all_points[:3]

    # if all([t in [i[0] for i in top4] for t in check_team]): 
    #     print(all_points)
    #     return True
    # return False

def prune_tree(root, points_dict, check_team):
    # print(check_team)
    point_list = []
    stack = [root]
    leaf_node = []
    leaf_scenes = []
    while stack:
        node = stack.pop()
        if not node.children:  # Leaf node
            leaf_node.append(node)
            rank = evaluate_condition(node, dict(points_dict), check_team)
            point_list.append(rank)

            leaf_scene = pd.DataFrame(data = list(node.winners)).T
            leaf_scenes.append(leaf_scene)
            # if not evaluate_condition(node, dict(points_dict), check_team):
            #     # Prune subtree by removing all children
            #     node.parent.children.remove(node)
            # else:
            #     win_list.append(node)
        else:
            stack.extend(node.children)

    point_dict = dict(Counter(point_list))
    point_dict = {k:v/len(point_list) for k,v in point_dict.items()}
    return leaf_scenes, point_list, point_dict

points_dict = points_table.set_index('TEAM_NAME').to_dict()['PTS']



team_points = {}
scenario = {}
# for t in mapping['Team Abbr'].unique():
for t in teams:
    print(t)
    leaf_scenes, point_list, point_dict = prune_tree(copy.deepcopy(root), points_dict, t)
    team_points[t] =  point_dict

    # scenario[t] = pd.DataFrame(columns = [f"{m.team1.name} v {m.team2.name}" for m in match_dict.values()])
    scenario[t] = pd.concat(leaf_scenes).reset_index(drop = True)
    scenario[t] = scenario[t].reset_index(drop=True).rename_axis('ID').reset_index().merge(pd.DataFrame(point_list).reset_index(drop=True).rename_axis('ID').reset_index(), on = 'ID', how = 'left')
    scenario[t].columns = ['ID'] + [f"{m.team1.name} v {m.team2.name}" for m in match_dict.values()] + ['RANK']

   

In [ ]:
# display(scenario[t][scenario[t]['RANK']>4])
t = 'DC'
display(scenario[t])
match_prob = {}
a = scenario[t][scenario[t]['RANK']<=4].copy()
for m in [f"{m.team1.name} v {m.team2.name}" for m in match_dict.values()]:
    v = a[m].value_counts(normalize = True).sort_values(ascending = True).rename_axis('MATCH').reset_index().T.reset_index()
    v.columns = v.iloc[0]
    v = v.iloc[1:,:]
    v = v.melt(id_vars = 'MATCH', var_name = 'WIN_TEAM', value_name = 'REQD_PERC').sort_values(by = 'REQD_PERC',ascending = False).iloc[:1,:]
    # .reset_index()
    match_prob[m] = v

final_scenario = pd.concat(match_prob.values()).reset_index(drop = True)


final_scenario['REQD_PERC'] = final_scenario['REQD_PERC'].fillna(0).apply(lambda x: f'{x:.2%}').T

# Define a function to apply conditional formatting
def highlight_cells(val):
    return f'background-color: rgba(0, 100, 255, {val})'  # Green color with alpha based on value

# Apply styling to the DataFrame
# final_scenario = final_scenario.style.apply(highlight_cells)

# Render styled DataFrame
final_scenario